In [131]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [132]:
if os.environ['GEMINI_API_KEY']:
    print("API Key loaded successfully.")

API Key loaded successfully.


In [133]:
#from langchain_openai import ChatOpenAI

In [134]:
#llm = ChatOpenAI(model_name="gpt-5-nano", temperature=0)

In [135]:
# response=llm.invoke("what is the capital of France?")
# response.content

In [136]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)



**RAG IMPLEMENTATION WITH OWN PDF**

step 1
extract text from pdf

In [137]:
from langchain_community.document_loaders import PyPDFLoader

In [138]:
pdf_path = r"C:\Users\hp\Downloads\ConfirmationPage-250310406970.pdf"

In [139]:
loader= PyPDFLoader(pdf_path)
docs=loader.load()

step 2 splitting the document into chunks

In [140]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)
chunks = splitter.split_documents(docs)
chunks

[Document(metadata={'producer': 'jsPDF 0.0.0', 'creator': 'PyPDF', 'creationdate': '2024-11-10T22:32:06+05:30', 'source': 'C:\\Users\\hp\\Downloads\\ConfirmationPage-250310406970.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Application Number : 250310406970   IP Address : 14.139.61.211                   Date and Time : 10/11/2024 10:32:06 PM               [Page  1 of 1 ]')]

In [141]:
len(chunks)

1

step3 creating embeddings for the chunks

In [142]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings( model="models/gemini-embedding-001")

step 4 create and store embeddings in vector store

In [143]:
from langchain_community.vectorstores import Chroma

In [144]:
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model
)

In [145]:
chunks[0]

Document(metadata={'producer': 'jsPDF 0.0.0', 'creator': 'PyPDF', 'creationdate': '2024-11-10T22:32:06+05:30', 'source': 'C:\\Users\\hp\\Downloads\\ConfirmationPage-250310406970.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Application Number : 250310406970   IP Address : 14.139.61.211                   Date and Time : 10/11/2024 10:32:06 PM               [Page  1 of 1 ]')

step 5 semantic search

In [146]:
 vector_store.similarity_search("what is application id? ",k=3)

[Document(metadata={'total_pages': 1, 'creator': 'PyPDF', 'producer': 'jsPDF 0.0.0', 'creationdate': '2024-11-10T22:32:06+05:30', 'source': 'C:\\Users\\hp\\Downloads\\ConfirmationPage-250310406970.pdf', 'page': 0, 'page_label': '1'}, page_content='Application Number : 250310406970   IP Address : 14.139.61.211                   Date and Time : 10/11/2024 10:32:06 PM               [Page  1 of 1 ]'),
 Document(metadata={'creationdate': '2024-11-10T22:32:06+05:30', 'total_pages': 1, 'creator': 'PyPDF', 'page_label': '1', 'page': 0, 'producer': 'jsPDF 0.0.0', 'source': 'C:\\Users\\hp\\Downloads\\ConfirmationPage-250310406970.pdf'}, page_content='Application Number : 250310406970   IP Address : 14.139.61.211                   Date and Time : 10/11/2024 10:32:06 PM               [Page  1 of 1 ]'),
 Document(metadata={'source': 'ABC', 'documentID': 'doc1'}, page_content='a Hindu-majority dominion of India and a Muslim-majority dominion of Pakistan. A large-scale loss of life and an unprecedent

talk to llm

In [147]:
context=vector_store.similarity_search("what is application id? ",k=3)

In [148]:
response=llm.invoke(f"what is application id ? Use the below context to answer the question. Context: {context}")
print(response.content)

Based on the context provided, the **Application Number** (which is equivalent to an application ID) is **250310406970**.
